In [1]:
%pip install -e git+https://github.com/saumya06/crewAI_nlsql.git@nlsql#egg=crewai
%cd '/home/wsuser/work/src/crewai'
%pip install .
%cd '/home/wsuser/work/'

Obtaining crewai from git+https://github.com/saumya06/crewAI_nlsql.git@nlsql#egg=crewai
  Updating ./src/crewai clone (to revision nlsql)
  Running command git fetch -q --tags
  Running command git reset --hard -q f0855c76257f5a048f223f237495da7ef389465c
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for crewai (pyproject.toml) ... done
  Created wheel for crewai: filename=crewai-0.108.0-py3-none-any.whl size=13487 sha256=c8e3ef3d376bdcf3e6d857a6ec2525bff0244271627b722d5f947941c86eb909
  Stored in directory: /tmp/wsuser/pip-ephem-wheel-cache-oi_jdvy6/wheels/67/83/0d/e3b4d3b1ad8b43c22af6bd46f5358ab90c520863e25c50f2ae
Successfully built crewai
  Attempting uninstall: crewai
    Found existing installation: crewai 0.108.0
    Uninstalling crewai-0.108.0:
      Successf

/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Processing /home/wsuser/work/src/crewai
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for crewai: filename=crewai-0.108.0-py3-none-any.whl size=265353 sha256=34e72f6197989ba68d1c3d25d3b17fd7a14b4fcce455e8168361b030e174c1f9
  Stored in directory: /tmp/wsuser/pip-ephem-wheel-cache-_qd13faq/wheels/45/0f/f7/41283362c60b2f77a5d953e3289bbeaa6bab5c024dbea49c79
Successfully built crewai
  Attempting uninstall: crewai
    Found existing installation: crewai 0.108.0
    Uninstalling crewai-0.108.0:
      Successfully uninstalled crewai-0.108.0
Note: you may need to restart the kernel to use updated packages.
/home/wsuser/work


In [ ]:
%pip install crewai[tools] 
from crewai_tools import NL2SQLTool
from crewai import LLM
import os
from crewai import Agent, Crew, Task, Process

/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:404: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.


In [ ]:
DB_URL = "cb0cfae7-372d-4047-9bf9-882289cd9910.blijti4d0v0nkr55oei0.databases.appdomain.cloud"
DB_PORT = "32240"
DB_NAME = "marcht3db"
DB_USER = "ibm_cloud_d2c0138f_5cb3_499e_8043_6a959fb57913"
DB_PASSWORD = input("Enter password for the user:") 

In [ ]:
nl2sql = NL2SQLTool(db_uri=f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_URL}:{DB_PORT}/{DB_NAME}")

Prepare your IBM Cloud API Key. Go to https://dataplatform.cloud.ibm.com/wx/home?context=wx > Developer Access > Create API key

In [ ]:
# Initialize LLM
llm = LLM(
    api_key="your_api_key",
    api_base = "https://us-south.ml.cloud.ibm.com",
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "sample",
        "max_new_tokens": 15000,
        "temperature": 0,
        "repetition_penalty": 1.05
    }
)

def create_dbagent() -> Agent:
    return Agent(
        role="Database Analyst",
        goal="""Generates the report based on user question and information in database. Always write the colum name as table_name."column_name" in query.  Formulate the respose in table structure.""",
        backstory="""You are an expert data analyst creating reports from database.""",
        llm=llm,
        allow_delegation=False,
        tools=[nl2sql]
    )


def create_generation_task(agent, natural_query):
    return Task(
        description=f"""{natural_query}.""",
        expected_output="""Response from database after running the query.""",
        agent=agent
    )

In [7]:
#Example Question 1: sales of Xtralife for last 3 months
#Example Question 2: Percentage change of monthly sales of Xtralife every month
user_question = input("Enter question:")

query_agent=create_dbagent()
generation_task = create_generation_task(query_agent, user_question)

generation_crew = Crew(
    agents=[query_agent],
    tasks=[generation_task],
    verbose=True
    )

crew_output = generation_crew.kickoff()
print ("Answer:...... \n" ,crew_output)

Enter question: sales of Xtralife for last 3 months


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8514b909-c9a0-4a92-b303-7205ff1cd206                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: d28d5418-167f-489a-9503-fc227093eec5
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: d28d5418-167f-489a-9503-fc227093eec5
       Status: Executing Task...
    └── 🤖 Agent: Database Analyst
            Status: In Progress

# Agent: Database Analyst
## Task: sales of Xtralife for last 3 months.


...

🚀 Crew: crew
└── 📋 Task: d28d5418-167f-489a-9503-fc227093eec5
       Status: Executing Task...
    └── 🤖 Agent: Database Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (5)

🚀 Crew: crew
└── 📋 Task: d28d5418-167f-489a-9503-fc227093eec5
       Assigned to: Database Analyst
       Status: ✅ Completed
    └── 🤖 Agent: Database Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (5)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d28d5418-167f-489a-9503-fc227093eec5                                                                     │
│  Agent: Database Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8514b909-c9a0-4a92-b303-7205ff1cd206                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Answer:...... 
 | OrderID      | OrderDate  | ProductName | Quantity |
|--------------|------------|-------------|----------|
| US-2014-106663| 2025-01-12 | Xtralife    | 700      |
| CA-2012-135272| 2025-02-23 | Xtralife    | 800      |
| AF-3014-153045| 2024-12-27 | Xtralife    | 2900     |


In [8]:
#Example Question 1: sales of Xtralife for last 3 months
#Example Question 2: Percentage change of monthly sales of Xtralife every month
user_question = input("Enter question:")

query_agent=create_dbagent()
generation_task = create_generation_task(query_agent, user_question)

generation_crew = Crew(
    agents=[query_agent],
    tasks=[generation_task],
    verbose=True
    )

crew_output = generation_crew.kickoff()
print ("Answer:...... \n" ,crew_output)

Enter question: Percentage change of monthly sales of Xtralife every month


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 033c050f-a29e-462b-a18a-f0942b5346ec                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f0294228-c6b9-40b9-8756-55cd7b8673fb
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f0294228-c6b9-40b9-8756-55cd7b8673fb
       Status: Executing Task...
    └── 🤖 Agent: Database Analyst
            Status: In Progress

# Agent: Database Analyst
## Task: Percentage change of monthly sales of Xtralife every month.


...

🚀 Crew: crew
└── 📋 Task: f0294228-c6b9-40b9-8756-55cd7b8673fb
       Status: Executing Task...
    └── 🤖 Agent: Database Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (10)

🚀 Crew: crew
└── 📋 Task: f0294228-c6b9-40b9-8756-55cd7b8673fb
       Assigned to: Database Analyst
       Status: ✅ Completed
    └── 🤖 Agent: Database Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (10)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f0294228-c6b9-40b9-8756-55cd7b8673fb                                                                     │
│  Agent: Database Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 033c050f-a29e-462b-a18a-f0942b5346ec                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Answer:...... 
 | OrderDate | ProductName | Sales | prev_sales | percentage_change |
| --- | --- | --- | --- | --- |
| 2024-03-14 | Xtralife | 190000.0 | 0.0 | NULL |
| 2024-04-15 | Xtralife | 200000.0 | 190000.0 | 5.263157894736842 |
| 2024-05-21 | Xtralife | 230000.0 | 200000.0 | 15.0 |
| 2024-06-26 | Xtralife | 260000.0 | 230000.0 | 13.043478260869565 |
| 2024-07-18 | Xtralife | 280000.0 | 260000.0 | 7.6923076923076925 |
| 2024-08-20 | Xtralife | 310000.0 | 280000.0 | 10.714285714285715 |
| 2024-09-22 | Xtralife | 330000.0 | 310000.0 | 6.451612903225806 |
| 2024-10-26 | Xtralife | 360000.0 | 330000.0 | 9.090909090909092 |
| 2024-11-28 | Xtralife | 390000.0 | 360000.0 | 8.333333333333333 |
| 2024-12-27 | Xtralife | 400000.0 | 390000.0 | 2.564102564102564 |
| 2025-01-12 | Xtralife | 110000.0 | 400000.0 | -72.5 |
| 2025-02-23 | Xtralife | 140000.0 | 110000.0 | 27.27272727272727 |
